In [15]:
# Import libraries
import numpy as np
from numpy import linalg as la

import random
import time
from IPython import display
import matplotlib.animation as animation
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True

import seaborn as sns

Backward substitution

In [16]:
def backwardsubstitution(U, b):
    # function which takes an upper triangular matrix U (size n x n) and vector b (size n) and returns vector x 
    # (size x) satisfying Ux = b
    n = len(b)
    x = np.zeros((n))
    x[-1] = b[-1]/U[-1,-1]
    for i in range(n-2,-1,-1):
        summ = 0
        for j in range(i+1,n):
            summ += U[i,j]*x[j]
        x[i] = (1/U[i,i])*(b[i] - summ)
    
    return x

In [17]:
U0 = np.matrix([[1,2,3],[0,1,1],[0,0,5]])
b0 = np.array([13,3,10])
x0 = backwardsubstitution(U0, b0)

x0

array([5., 1., 2.])

Forward substitution

In [18]:
def forwardsubstitution(L, b):
    # function which takes a lower triangular matrix L (size n x n) and vector b (size n) and returns vector x 
    # (size x) satisfying Lx = b
    n = len(b)
    x = np.zeros((n))
    x[0] = b[0]/L[0,0]
    for i in range(1,n):
        summ = 0
        for j in range(0,i):
            summ += L[i,j]*x[j]
        x[i] = (1/L[i,i])*(b[i] - summ)
    
    return x

In [19]:
L = np.array([[5,0,0],[2,3,0],[1,-2,4]])
b = np.array([10,19,4])
x0 = forwardsubstitution(L, b)

x0

array([2., 5., 3.])

Crout's algorithm for LU decomposition

In [20]:
def Crout(A):
    # function which takes in nxn matrix A and evaluates its LU decomposition
    # NB: without partial pivoting (for now)
    n = len(A[:,0])
    
    L = np.identity(n)
    U = np.zeros((n,n))
    
    for j in range(0,n): # loop through columns
        for i in range(j+1):
            U[i,j] = A[i,j] - np.sum([L[i,k]*U[k,j] for k in range(i)])
        for i in range(j+1,n):
            L[i,j] = (A[i,j] - np.sum([L[i,k]*U[k,j] for k in range(j)]))/U[j,j]
            
    return L, U

In [21]:
testA = np.array([[1, 2, 4], [3, 8, 14], [2, 6, 13]])

decomp = Crout(testA)

In [22]:
decomp

(array([[1., 0., 0.],
        [3., 1., 0.],
        [2., 1., 1.]]),
 array([[1., 2., 4.],
        [0., 2., 2.],
        [0., 0., 3.]]))

In [24]:
# checking the invertibility of the submatrices of A
n = 3
for j in range(1,n+1):
    print(testA[:j,:j])
    print(la.det(testA[:j,:j]))
    print('\n')

[[1]]
1.0


[[1 2]
 [3 8]]
1.9999999999999998


[[ 1  2  4]
 [ 3  8 14]
 [ 2  6 13]]
6.0




In [90]:
# partial pivoting
def PivotCrout(A):
    # function which takes in nxn matrix A and evaluates its LU decomposition
    # NB: without partial pivoting (for now)
    n = len(A[:,0])
    print(A)
    L = np.identity(n)
    U = np.zeros((n,n))
    P = np.identity(n)

    for j in range(1,n):
        print(j)
        print(A[:j,:j])
        print('\n')
        if la.det(A[:j,:j]) == 0:
            rowswap = np.argmax([A[k,k] for k in range(j,n)])
            P_new = np.identity(n)
            P_new[j-1,j-1] = 0
            P_new[j+rowswap,j+rowswap] = 0
            P_new[j+rowswap,j-1] = 1
            P_new[j-1,j+rowswap] = 1
            A = np.matmul(P_new,A)
            P = np.matmul(P_new,P)
            print(P_new)
    
    for j in range(0,n): # loop through columns
        for i in range(j+1):
            U[i,j] = A[i,j] - np.sum([L[i,k]*U[k,j] for k in range(i)])
        for i in range(j+1,n):
            L[i,j] = (A[i,j] - np.sum([L[i,k]*U[k,j] for k in range(j)]))/U[j,j]
            
    return np.array([P, L, U])

In [91]:
testA = np.random.randint(-10,10,size=(10,10))
singularA = np.array([[1,2,2,-1],[1,2,2,-1],[3,2,-1,6],[5,2,7,-3]])

decomp = PivotCrout(singularA)

[[ 1  2  2 -1]
 [ 1  2  2 -1]
 [ 3  2 -1  6]
 [ 5  2  7 -3]]
1
[[1]]


2
[[1 2]
 [1 2]]


[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
3
[[ 1.  2.  2.]
 [ 3.  2. -1.]
 [ 1.  2.  2.]]


[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


In [93]:
P = decomp[0,:,:]
L = decomp[1,:,:]
U = decomp[2,:,:]

print(P@singularA)
print(L@U)

[[ 1.  2.  2. -1.]
 [ 3.  2. -1.  6.]
 [ 5.  2.  7. -3.]
 [ 1.  2.  2. -1.]]
[[ 1.  2.  2. -1.]
 [ 3.  2. -1.  6.]
 [ 5.  2.  7. -3.]
 [ 1.  2.  2. -1.]]


In [ ]:
np.shape()